# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**:David Abraham Naranjo Salgado

**Professor**: Pablo Camarillo Ramirez

In [38]:
import findspark
findspark.init()

In [39]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-David-Naranjo") \
    .master("spark://a343afeb5c7a:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [40]:
import importlib
import team_name.spark_utils

In [41]:

importlib.reload(team_name.spark_utils)

columns_info = [ ("department_id", "string"),
                ("department_name", "string"),
                ("location", "string")]

schema = team_name.spark_utils.SparkUtils.generate_schema(columns_info)

departments_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/departments.csv")
                
departments_df.show(10)

+-------------+--------------------+-------------+
|department_id|     department_name|     location|
+-------------+--------------------+-------------+
|          101|     Human Resources|    San Diego|
|          102|Finance and Accou...|     New York|
|          103| Sales and Marketing|      Chicago|
|          104|    Data Engineering|      Zapopan|
|          105|        Data Science|      Seattle|
|          106|          Operations|       London|
|          107|               Legal|      Chicago|
|          108|Research and Deve...| Philadelphia|
|          109|    Customer Service|San Francisco|
|          110|Corporate Strateg...|  Los Angeles|
+-------------+--------------------+-------------+



In [42]:

importlib.reload(team_name.spark_utils)

columns_info = [ ("employee_id", "string"),
                ("employee_info", "string")]

schema = team_name.spark_utils.SparkUtils.generate_schema(columns_info)

employee_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/employees.csv")
                
employee_df.show(10)

+-----------+--------------------+
|employee_id|       employee_info|
+-----------+--------------------+
|          1|{'name': 'Caitlyn...|
|          2|{'name': 'Rachel'...|
|          3|{'name': 'Carrie'...|
|          4|{'name': 'Renee',...|
|          5|{'name': 'Gabriel...|
|          6|{'name': 'Kristen...|
|          7|{'name': 'Jonatha...|
|          8|{'name': 'Michell...|
|          9|{'name': 'Michell...|
|         10|{'name': 'Lisa', ...|
+-----------+--------------------+
only showing top 10 rows



In [43]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

In [44]:

employee_info_schema = StructType([
    StructField("name", StringType(), True),
    StructField("department_id", IntegerType(), True),
    StructField("salary", DoubleType(), True),
    StructField("hire_date", DateType(), True)
])

employee_df_parsed = employee_df.withColumn("employee_info_parsed", from_json(col("employee_info"), employee_info_schema))

employee_df_selected = employee_df_parsed.select(
    "employee_id",
    "employee_info_parsed.name",
    "employee_info_parsed.department_id",
    "employee_info_parsed.salary",
    "employee_info_parsed.hire_date"
)

employee_df_selected.show(10)


+-----------+---------+-------------+---------+----------+
|employee_id|     name|department_id|   salary| hire_date|
+-----------+---------+-------------+---------+----------+
|          1|  Caitlyn|          103|115959.78|2002-06-10|
|          2|   Rachel|          104|100820.16|2009-07-01|
|          3|   Carrie|          105|114421.44|1998-12-10|
|          4|    Renee|          104| 54688.13|1995-03-17|
|          5|Gabriella|          109|106267.03|1995-02-09|
|          6|  Kristen|          101| 88237.54|2010-11-15|
|          7| Jonathan|          102| 39323.42|2012-06-30|
|          8| Michelle|          101| 64262.85|2005-10-30|
|          9| Michelle|          105|103521.88|1991-07-10|
|         10|     Lisa|          110| 55435.93|2016-03-25|
+-----------+---------+-------------+---------+----------+
only showing top 10 rows



In [45]:
employee_combined_df = employee_df_selected.join(
    departments_df, 
    employee_df_selected.department_id == departments_df.department_id,
    "inner"
)
employee_combined_df.show(5)

+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+
|employee_id|     name|department_id|   salary| hire_date|department_id|    department_name|     location|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+
|          1|  Caitlyn|          103|115959.78|2002-06-10|          103|Sales and Marketing|      Chicago|
|          2|   Rachel|          104|100820.16|2009-07-01|          104|   Data Engineering|      Zapopan|
|          3|   Carrie|          105|114421.44|1998-12-10|          105|       Data Science|      Seattle|
|          4|    Renee|          104| 54688.13|1995-03-17|          104|   Data Engineering|      Zapopan|
|          5|Gabriella|          109|106267.03|1995-02-09|          109|   Customer Service|San Francisco|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+
only showing top 5 rows



In [46]:
from pyspark.sql.functions import when
employee_combined_df = employee_combined_df.withColumn(
    "salary_category",
    when(col("salary") >= 55000, "High").otherwise("Low")
)

employee_combined_df.show(5)

+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|employee_id|     name|department_id|   salary| hire_date|department_id|    department_name|     location|salary_category|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|          1|  Caitlyn|          103|115959.78|2002-06-10|          103|Sales and Marketing|      Chicago|           High|
|          2|   Rachel|          104|100820.16|2009-07-01|          104|   Data Engineering|      Zapopan|           High|
|          3|   Carrie|          105|114421.44|1998-12-10|          105|       Data Science|      Seattle|           High|
|          4|    Renee|          104| 54688.13|1995-03-17|          104|   Data Engineering|      Zapopan|            Low|
|          5|Gabriella|          109|106267.03|1995-02-09|          109|   Customer Service|San Francisco|           High|
+-----------+---

In [47]:
from pyspark.sql.functions import avg

In [48]:
high_salary_df = employee_combined_df.filter(
    col("salary_category") == "High"
)
high_salary_df.show(5)

+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|employee_id|     name|department_id|   salary| hire_date|department_id|    department_name|     location|salary_category|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|          1|  Caitlyn|          103|115959.78|2002-06-10|          103|Sales and Marketing|      Chicago|           High|
|          2|   Rachel|          104|100820.16|2009-07-01|          104|   Data Engineering|      Zapopan|           High|
|          3|   Carrie|          105|114421.44|1998-12-10|          105|       Data Science|      Seattle|           High|
|          5|Gabriella|          109|106267.03|1995-02-09|          109|   Customer Service|San Francisco|           High|
|          6|  Kristen|          101| 88237.54|2010-11-15|          101|    Human Resources|    San Diego|           High|
+-----------+---

In [49]:
low_salary_df = employee_combined_df.filter(
    col("salary_category") == "Low"
)
low_salary_df.show(5)

+-----------+--------+-------------+--------+----------+-------------+--------------------+-------------+---------------+
|employee_id|    name|department_id|  salary| hire_date|department_id|     department_name|     location|salary_category|
+-----------+--------+-------------+--------+----------+-------------+--------------------+-------------+---------------+
|          4|   Renee|          104|54688.13|1995-03-17|          104|    Data Engineering|      Zapopan|            Low|
|          7|Jonathan|          102|39323.42|2012-06-30|          102|Finance and Accou...|     New York|            Low|
|         13|    Lisa|          104|36032.49|2019-05-16|          104|    Data Engineering|      Zapopan|            Low|
|         26|    John|          109|44836.57|2004-11-13|          109|    Customer Service|San Francisco|            Low|
|         38|  Rachel|          109|43269.85|1992-07-13|          109|    Customer Service|San Francisco|            Low|
+-----------+--------+--

In [50]:
high_salary_avg = high_salary_df.groupBy("department_name") \
    .agg(avg("salary").alias("avg_salary"))
high_salary_avg.show()

+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|102741.38324414717|
| Sales and Marketing|100839.65275449108|
|    Data Engineering|101626.29492163012|
|Research and Deve...|  98714.3003086419|
|Finance and Accou...|100731.07877887784|
|    Customer Service|101585.01600000002|
|               Legal|  99366.3129102167|
|        Data Science|101903.63710344829|
|          Operations|100169.65621722837|
|     Human Resources|104999.43191489363|
+--------------------+------------------+



In [51]:
low_salary_avg = low_salary_df.groupBy("department_name") \
    .agg(avg("salary").alias("avg_salary"))
low_salary_avg.show()

+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|41590.741833333326|
| Sales and Marketing| 41150.40277777778|
|    Data Engineering| 41358.50794117647|
|Finance and Accou...|42740.952888888874|
|Research and Deve...| 41426.43521126761|
|    Customer Service|42644.472021276604|
|               Legal| 41160.26616438357|
|        Data Science| 41974.18958333334|
|          Operations|40646.100705882345|
|     Human Resources| 41751.64784810126|
+--------------------+------------------+



In [52]:
top_high_salary= high_salary_df \
    .orderBy(col("salary").desc()) \
    .limit(5)
top_high_salary.show()

+-----------+---------+-------------+---------+----------+-------------+--------------------+-----------+---------------+
|employee_id|     name|department_id|   salary| hire_date|department_id|     department_name|   location|salary_category|
+-----------+---------+-------------+---------+----------+-------------+--------------------+-----------+---------------+
|       1778|Gabriella|          101|149989.73|2018-09-14|          101|     Human Resources|  San Diego|           High|
|       3621|Katherine|          101| 149979.3|2017-07-26|          101|     Human Resources|  San Diego|           High|
|        346|     Ryan|          110| 149963.1|1990-07-03|          110|Corporate Strateg...|Los Angeles|           High|
|       3807|  Caitlyn|          107|149956.54|2000-07-27|          107|               Legal|    Chicago|           High|
|       3050|     Mark|          107|149915.56|2007-11-06|          107|               Legal|    Chicago|           High|
+-----------+---------+-

In [53]:
top_low_salary = low_salary_df \
    .orderBy(col("salary").desc()) \
    .limit(5)
top_low_salary.show()

+-----------+-----+-------------+--------+----------+-------------+--------------------+-------------+---------------+
|employee_id| name|department_id|  salary| hire_date|department_id|     department_name|     location|salary_category|
+-----------+-----+-------------+--------+----------+-------------+--------------------+-------------+---------------+
|       3472|Linda|          110|54993.53|2017-01-15|          110|Corporate Strateg...|  Los Angeles|            Low|
|       2545|Tammy|          104|54991.71|2004-12-07|          104|    Data Engineering|      Zapopan|            Low|
|        382|Aaron|          102|54989.45|2011-03-20|          102|Finance and Accou...|     New York|            Low|
|       2153|Craig|          101| 54945.2|2016-04-24|          101|     Human Resources|    San Diego|            Low|
|       3024|Aaron|          109| 54937.3|1994-06-25|          109|    Customer Service|San Francisco|            Low|
+-----------+-----+-------------+--------+------

In [54]:
from pyspark.sql.functions import current_date, datediff

In [55]:
employee_df_with_years = employee_df_selected.withColumn(
    "years_in_company",
    (datediff(current_date(), col("hire_date")) / 365).cast("integer")
)
employee_df_with_years.show(5)

+-----------+---------+-------------+---------+----------+----------------+
|employee_id|     name|department_id|   salary| hire_date|years_in_company|
+-----------+---------+-------------+---------+----------+----------------+
|          1|  Caitlyn|          103|115959.78|2002-06-10|              22|
|          2|   Rachel|          104|100820.16|2009-07-01|              15|
|          3|   Carrie|          105|114421.44|1998-12-10|              26|
|          4|    Renee|          104| 54688.13|1995-03-17|              30|
|          5|Gabriella|          109|106267.03|1995-02-09|              30|
+-----------+---------+-------------+---------+----------+----------------+
only showing top 5 rows



In [56]:
employees_more_than_5_years = employee_df_with_years.filter(col("years_in_company") > 5)
num_employees_more_than_5_years = employees_more_than_5_years.count()
print(num_employees_more_than_5_years)

3225


In [57]:
sc.stop()